# Objetivos de la práctica

En esta actividad propone desarrollar un proyecto de Ciencia de Datos respecto a el uso de bicicletas por parte de una empresa.

In [1]:
import pandas as pd
import os
print(os.getcwd())
os.chdir('./../')
print(os.getcwd())
import matplotlib.pyplot as plt

d:\Kike\UNI\1º\Introduccion a la ciencia de datos\Practicas\notebooks
d:\Kike\UNI\1º\Introduccion a la ciencia de datos\Practicas


# Carga y análisis de datos

Lo primero es comprender el problema desde la fuente de datos original


## Descarga de uso de bicis

https://valencia.opendatasoft.com/pages/home/

https://valencia.opendatasoft.com/explore/dataset/valenbisi-disponibilitat-valenbisi-dsiponibilidad/table/?flg=es-es


In [2]:
dfb = pd.read_csv('./data/raw/usobici_raw.csv')
dfb.tail()

,updated,address,number,available,lat,lon,total,open,free
4063,2025-03-03 14:39:32.000000,Alameda - Pintor Maella,53,24,39.456764,-0.348139,25,T,1
4064,2025-03-03 14:49:38.000000,Alameda - Pintor Maella,53,24,39.456764,-0.348139,25,T,1
4065,2025-03-03 15:00:44.000000,Alameda - Pintor Maella,53,24,39.456764,-0.348139,25,T,1
4066,2025-03-03 15:09:50.000000,Alameda - Pintor Maella,53,23,39.456764,-0.348139,25,T,2
4067,2025-03-03 15:19:57.000000,Alameda - Pintor Maella,53,23,39.456764,-0.348139,25,T,2


In [3]:
dfb.shape

(4068, 9)

In [6]:
dfb.describe(include='all')

,updated,address,number,available,lat,lon,total,open,free
count,4068,4068,4068.0,4068.000000,4.068000e+03,4.068000e+03,4068.0,4068,4068.000000
unique,4068,1,NaN,NaN,NaN,NaN,NaN,1,NaN
top,2025-03-03 15:19:57.000000,Alameda - Pintor Maella,NaN,NaN,NaN,NaN,NaN,T,NaN
freq,1,4068,NaN,NaN,NaN,NaN,NaN,4068,NaN
mean,NaN,NaN,53.0,17.118240,3.945676e+01,-3.481393e-01,25.0,NaN,7.855457
std,NaN,NaN,0.0,7.428375,7.106301e-15,5.551798e-17,0.0,NaN,7.442906
min,NaN,NaN,53.0,0.000000,3.945676e+01,-3.481393e-01,25.0,NaN,0.000000
25%,NaN,NaN,53.0,11.000000,3.945676e+01,-3.481393e-01,25.0,NaN,2.000000
50%,NaN,NaN,53.0,20.000000,3.945676e+01,-3.481393e-01,25.0,NaN,5.000000
75%,NaN,NaN,53.0,23.000000,3.945676e+01,-3.481393e-01,25.0,NaN,14.000000


In [4]:
# Tipos de datos en pandas
dfb.dtypes

updated       object
address       object
number         int64
available      int64
lat          float64
lon          float64
total          int64
open          object
free           int64
dtype: object

In [5]:
# Conversión de la columna fecha
dfb["updated"] = pd.to_datetime(dfb.updated)
# Rango temporal de la muestra
print(dfb["updated"].min(), dfb["updated"].max())

2025-02-03 00:01:42 2025-03-03 15:19:57


In [12]:
dfb["updated"]
dfb["updated"].dt.hour

0        0
1        0
2        0
3        0
4        0
        ..
4063    14
4064    14
4065    15
4066    15
4067    15
Name: updated, Length: 4068, dtype: int32

## Normalización de datos
- ¿Existen datos que se repiten?
- ¿Cómo podría afectar a nuestro proyecto la redundancia?
- ¿Qué acciones se podrían hacer para evitar problemas en un proyecto CD?
- Tras definir acciones, ¿altera nuestro proceso de exploración de los datos?

In [13]:
dfb["address"].unique().shape[0]

1

## Datos de estaciones

In [14]:
dfe = pd.read_csv('./data/interim/estaciones.csv')
dfe.tail()

,number,uso_bici,address,lat,lon,total,nombre,coddistrit,codbarrio
268,272,673,Vicente la Roda - Ingeniero Fausto Elio,39.480623,-0.332190,15,LA MALVA-ROSA,11,3
269,273,340,Moraira - Alta del Mar,39.450305,-0.333272,15,NATZARET,11,5
270,274,216,San Francisco de Paula - Castell de Pop,39.448070,-0.333188,15,NATZARET,11,5
271,275,418,Moreras - Rona de Nazaret,39.452305,-0.335065,20,LA PUNTA,10,6
272,276,1029,Veles e Vents,39.461975,-0.323765,20,EL GRAU,11,1


## Descarga de AEMET

- Hipótesis del problema planteado: las condiciones meteorológicas pueden condicionar al usuario para alquilar bicicletas


https://opendata.aemet.es/centrodedescargas/inicio

https://opendata.aemet.es/opendata/api/observacion/convencional/todas

aemet_metadata.json


In [15]:
dfm = pd.read_csv('./data/external/aemet.csv')
dfm.tail()

,ubi,prec,vmax,vv,hr,ta,tpr,tamin,tamax,fecha
610,VALENCIA DT,0.0,0.0,0.0,64.0,10.1,3.6,9.9,10.2,2025-02-28 05:00:00+00:00
611,VALENCIA DT,0.0,0.0,0.0,64.0,10.4,3.9,9.9,10.4,2025-02-28 06:00:00+00:00
612,VALENCIA DT,0.0,0.0,0.0,65.0,10.8,4.5,10.3,10.8,2025-02-28 07:00:00+00:00
613,VALENCIA DT,0.0,5.1,1.7,67.0,14.1,8.1,12.5,14.3,2025-02-28 09:00:00+00:00
614,VALENCIA DT,0.0,6.6,1.3,69.0,14.8,9.2,14.1,14.9,2025-02-28 10:00:00+00:00


In [57]:
dfm.sample(10)

,ubi,prec,vmax,vv,hr,ta,tpr,tamin,tamax,fecha
534,VALENCIA DT,0.0,0.0,0.0,31.0,19.3,1.7,19.3,20.6,2025-02-24 17:00:00+00:00
259,VALENCIA DT,0.0,4.1,1.5,60.0,10.3,2.9,10.2,10.9,2025-02-13 05:00:00+00:00
334,VALENCIA DT,0.0,0.0,0.0,73.0,12.2,7.6,10.1,12.3,2025-02-16 08:00:00+00:00
121,VALENCIA DT,0.0,4.0,0.7,40.0,15.8,2.3,14.2,15.9,2025-02-07 11:00:00+00:00
176,VALENCIA DT,0.0,3.2,1.7,64.0,12.6,5.9,12.4,13.4,2025-02-09 18:00:00+00:00
316,VALENCIA DT,0.0,0.0,0.0,67.0,17.0,10.9,16.3,17.9,2025-02-15 14:00:00+00:00
385,VALENCIA DT,0.0,0.0,0.0,56.0,16.8,8.0,16.0,17.2,2025-02-18 11:00:00+00:00
1,VALENCIA DT,0.0,6.5,2.2,38.0,14.3,0.2,12.8,14.5,2025-02-02 11:00:00+00:00
4,VALENCIA DT,0.0,4.8,1.9,40.0,13.7,0.4,13.5,14.8,2025-02-02 14:00:00+00:00
171,VALENCIA DT,0.0,3.3,0.7,40.0,16.2,2.7,15.8,17.1,2025-02-09 13:00:00+00:00


In [16]:
dfm.shape

(615, 10)

In [58]:
dfm.describe(include='all')

,ubi,prec,vmax,vv,hr,ta,tpr,tamin,tamax,fecha
count,615,615.000000,615.000000,615.000000,615.000000,615.00000,615.000000,615.000000,615.000000,615
unique,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,VALENCIA DT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,0.033496,1.589756,0.510244,62.497561,13.16374,5.693333,12.686667,13.668780,2025-02-15 05:32:11.707316992+00:00
min,NaN,0.000000,0.000000,0.000000,26.000000,5.40000,-1.700000,5.300000,5.600000,2025-02-02 10:00:00+00:00
25%,NaN,0.000000,0.000000,0.000000,52.000000,11.10000,2.900000,10.550000,11.600000,2025-02-08 19:30:00+00:00
50%,NaN,0.000000,0.000000,0.000000,66.000000,13.30000,5.900000,12.900000,13.700000,2025-02-15 05:00:00+00:00
75%,NaN,0.000000,3.100000,1.000000,75.000000,15.40000,8.850000,14.900000,15.900000,2025-02-21 14:30:00+00:00
max,NaN,15.600000,19.600000,4.500000,81.000000,22.10000,11.300000,21.300000,22.700000,2025-02-28 10:00:00+00:00


In [18]:
dfm.dtypes

ubi       object
prec     float64
vmax     float64
vv       float64
hr       float64
ta       float64
tpr      float64
tamin    float64
tamax    float64
fecha     object
dtype: object

In [19]:
# Conversión de la columna updated
dfm["fecha"] = pd.to_datetime(dfm.fecha)
# Rango temporal de la muestra
print(dfm["fecha"].min(), dfm["fecha"].max())

2025-02-02 10:00:00+00:00 2025-02-28 10:00:00+00:00


In [20]:
dfm["ubi"].unique().shape[0]

1

In [21]:
# Búsqueda de datos faltantes
sert = pd.date_range(start=dfm['fecha'].min(), 
                     end=dfm['fecha'].max(), freq='h')
data = {
    'datetime': sert,
    'value': range(len(sert))
}
df = pd.DataFrame(data)
full_range_df = pd.DataFrame(sert, columns=['fecha'])
merged_df = full_range_df.merge(dfm, on='fecha', how='left')
missing_data = merged_df[merged_df['tamin'].isna()]
print("Registros faltantes:")
print(missing_data)

Registros faltantes:
                        fecha  ubi  prec  vmax  vv  hr  ta  tpr  tamin  tamax
531 2025-02-24 13:00:00+00:00  NaN   NaN   NaN NaN NaN NaN  NaN    NaN    NaN
578 2025-02-26 12:00:00+00:00  NaN   NaN   NaN NaN NaN NaN  NaN    NaN    NaN
579 2025-02-26 13:00:00+00:00  NaN   NaN   NaN NaN NaN NaN  NaN    NaN    NaN
590 2025-02-27 00:00:00+00:00  NaN   NaN   NaN NaN NaN NaN  NaN    NaN    NaN
591 2025-02-27 01:00:00+00:00  NaN   NaN   NaN NaN NaN NaN  NaN    NaN    NaN
592 2025-02-27 02:00:00+00:00  NaN   NaN   NaN NaN NaN NaN  NaN    NaN    NaN
593 2025-02-27 03:00:00+00:00  NaN   NaN   NaN NaN NaN NaN  NaN    NaN    NaN
594 2025-02-27 04:00:00+00:00  NaN   NaN   NaN NaN NaN NaN  NaN    NaN    NaN
595 2025-02-27 05:00:00+00:00  NaN   NaN   NaN NaN NaN NaN  NaN    NaN    NaN
622 2025-02-28 08:00:00+00:00  NaN   NaN   NaN NaN NaN NaN  NaN    NaN    NaN


# Integración de datos

Los datos tienen una muestra que es una selección de estaciones del barrio de Ruzafa.

In [80]:
dfb = pd.read_csv('./data/interim/usoestameteo.csv')
dfb.head()
#dfb.tail()

,prec,vmax,vv,hr,ta,tpr,tamin,tamax,fecha,count_out,count_in,number,address,lat,lon,total,nombre,coddistrit,codbarrio
0,0.0,2.5,0.9,72.0,6.5,1.9,6.4,7.1,2025-02-03 00:00:00+00:00,0.0,0.0,148.0,C/Xàtiva 30-32,39.467145,-0.375378,35.0,RUSSAFA,2,1
1,0.0,2.1,0.5,71.0,6.3,1.5,6.2,6.6,2025-02-03 01:00:00+00:00,0.0,0.0,148.0,C/Xàtiva 30-32,39.467145,-0.375378,35.0,RUSSAFA,2,1
2,0.0,2.5,1.1,71.0,6.0,1.1,6.0,6.3,2025-02-03 02:00:00+00:00,0.0,0.0,148.0,C/Xàtiva 30-32,39.467145,-0.375378,35.0,RUSSAFA,2,1
3,0.0,2.6,0.4,70.0,5.8,0.9,5.8,6.2,2025-02-03 03:00:00+00:00,0.0,0.0,148.0,C/Xàtiva 30-32,39.467145,-0.375378,35.0,RUSSAFA,2,1
4,0.0,2.1,0.5,70.0,5.7,0.6,5.7,5.9,2025-02-03 04:00:00+00:00,0.0,0.0,148.0,C/Xàtiva 30-32,39.467145,-0.375378,35.0,RUSSAFA,2,1


In [23]:
# Estructura del DF
dfb.shape, dfb.columns

((4711, 19),
 Index(['prec', 'vmax', 'vv', 'hr', 'ta', 'tpr', 'tamin', 'tamax', 'fecha',
        'count_out', 'count_in', 'number', 'address', 'lat', 'lon', 'total',
        'nombre', 'coddistrit', 'codbarrio'],
       dtype='object'))

In [24]:
dfb.head()

,prec,vmax,vv,hr,ta,tpr,tamin,tamax,fecha,count_out,count_in,number,address,lat,lon,total,nombre,coddistrit,codbarrio
0,0.0,2.5,0.9,72.0,6.5,1.9,6.4,7.1,2025-02-03 00:00:00+00:00,0.0,0.0,148.0,C/Xàtiva 30-32,39.467145,-0.375378,35.0,RUSSAFA,2,1
1,0.0,2.1,0.5,71.0,6.3,1.5,6.2,6.6,2025-02-03 01:00:00+00:00,0.0,0.0,148.0,C/Xàtiva 30-32,39.467145,-0.375378,35.0,RUSSAFA,2,1
2,0.0,2.5,1.1,71.0,6.0,1.1,6.0,6.3,2025-02-03 02:00:00+00:00,0.0,0.0,148.0,C/Xàtiva 30-32,39.467145,-0.375378,35.0,RUSSAFA,2,1
3,0.0,2.6,0.4,70.0,5.8,0.9,5.8,6.2,2025-02-03 03:00:00+00:00,0.0,0.0,148.0,C/Xàtiva 30-32,39.467145,-0.375378,35.0,RUSSAFA,2,1
4,0.0,2.1,0.5,70.0,5.7,0.6,5.7,5.9,2025-02-03 04:00:00+00:00,0.0,0.0,148.0,C/Xàtiva 30-32,39.467145,-0.375378,35.0,RUSSAFA,2,1


In [25]:
# Conversión de la columna updated
dfb["fecha"] = pd.to_datetime(dfb.fecha)
# Rango temporal de la muestra
print(dfb["fecha"].min(), dfb["fecha"].max())

2025-02-03 00:00:00+00:00 2025-03-03 10:00:00+00:00


In [26]:
dfb.sort_values("fecha", ascending=True, inplace=True)
dfb.tail()

,prec,vmax,vv,hr,ta,tpr,tamin,tamax,fecha,count_out,count_in,number,address,lat,lon,total,nombre,coddistrit,codbarrio
1345,1.2,9.7,2.9,81.0,14.4,11.2,14.2,14.4,2025-03-03 10:00:00+00:00,-1.0,4.0,33.0,Germanías - Ruzafa,39.464818,-0.373990,20.0,RUSSAFA,2,1
2018,1.2,9.7,2.9,81.0,14.4,11.2,14.2,14.4,2025-03-03 10:00:00+00:00,0.0,0.0,38.0,Peris y Valero - Cabo Jubi,39.459402,-0.370249,15.0,RUSSAFA,2,1
2691,1.2,9.7,2.9,81.0,14.4,11.2,14.2,14.4,2025-03-03 10:00:00+00:00,-1.0,0.0,39.0,Peris y Valero - Cuba,39.457854,-0.373931,19.0,RUSSAFA,2,1
3364,1.2,9.7,2.9,81.0,14.4,11.2,14.2,14.4,2025-03-03 10:00:00+00:00,-3.0,7.0,37.0,Peris y Valero - Luis Santángel,39.460938,-0.366441,20.0,RUSSAFA,2,1
4710,1.2,9.7,2.9,81.0,14.4,11.2,14.2,14.4,2025-03-03 10:00:00+00:00,-1.0,2.0,152.0,Reina Doña María - Cádiz,39.460925,-0.372705,15.0,RUSSAFA,2,1


In [27]:
# Visualizar datos nulos por columnas
dfb.isna().sum()

prec          0
vmax          0
vv            0
hr            0
ta            0
tpr           0
tamin         0
tamax         0
fecha         0
count_out     0
count_in      0
number        0
address       0
lat           0
lon           0
total         0
nombre        0
coddistrit    0
codbarrio     0
dtype: int64

In [59]:
# Identificar las filas que tienen valores nulos 
# y conocer su distribución en la serie revisar variables: 
# count_out, address
dfb.loc[dfb['nombre'].isna(), ['fecha', 'count_out', 'nombre']]

,fecha,count_out,nombre


# Eliminar valores nulos

In [60]:
# No lo hacemos persistente el cambio, 
# solo muestra el df total resultado tras eliminar
dfb.dropna(how='any')['nombre'].value_counts().sort_index()

nombre
RUSSAFA    4711
Name: count, dtype: int64

In [31]:
# en caso de haber nulos eliminaría los registros
df2 = dfb.dropna(how='any').copy()
df2.shape

(4711, 19)

In [32]:
df2.isna().sum()

prec          0
vmax          0
vv            0
hr            0
ta            0
tpr           0
tamin         0
tamax         0
fecha         0
count_out     0
count_in      0
number        0
address       0
lat           0
lon           0
total         0
nombre        0
coddistrit    0
codbarrio     0
dtype: int64

# Análisis exploratorio

In [33]:
df2 = pd.read_csv('./data/interim/usobarriosmeteo.csv')
df2.tail()

,prec,vmax,vv,hr,ta,tpr,tamin,tamax,fecha,uso_bici,name
47105,0.0,8.6,2.0,81.0,14.1,10.9,13.7,14.1,2025-03-03 06:00:00+00:00,1.0,SOTERNES
47106,0.0,9.2,2.1,81.0,14.3,11.1,14.1,14.4,2025-03-03 07:00:00+00:00,2.0,SOTERNES
47107,0.6,9.3,2.2,81.0,14.1,10.9,14.1,14.4,2025-03-03 08:00:00+00:00,0.0,SOTERNES
47108,1.5,8.0,2.3,81.0,14.3,11.1,14.0,14.4,2025-03-03 09:00:00+00:00,1.0,SOTERNES
47109,1.2,9.7,2.9,81.0,14.4,11.2,14.2,14.4,2025-03-03 10:00:00+00:00,0.0,SOTERNES


In [34]:
df2['name'].value_counts()

name
LA CARRASCA            673
SANT PAU               673
CABANYAL-CANYAMELAR    673
BENICALAP              673
ARRANCAPINS            673
                      ... 
LES TENDETES           673
SANT ANTONI            673
JAUME ROIG             673
LA PUNTA               673
SOTERNES               673
Name: count, Length: 70, dtype: int64

In [36]:
var_analisis = "name"

In [37]:
# Número de estaciones disponibles
# Estaciones únicas
df2[var_analisis].unique()
# Número de estaciones disponibles
# df2[var_analisis].unique().shape[0]

array(['LA CARRASCA', 'SANT PAU', 'CABANYAL-CANYAMELAR', 'BENICALAP',
       'ARRANCAPINS', 'NOU MOLES', 'RUSSAFA', 'AIORA', 'CAMPANAR',
       'MALILLA', 'CIUTAT DE LES ARTS I DE LES CIENCIES', 'LA GRAN VIA',
       'TORREFIEL', 'PENYA-ROJA', 'EL PLA DEL REMEI', 'SANT FRANCESC',
       'MESTALLA', 'BENIMACLET', 'EL CARME', 'MONTOLIVET',
       'CIUTAT UNIVERSITARIA', 'LA MALVA-ROSA', 'EXPOSICIO', 'ALBORS',
       'SANT LLORENS', 'MARXALENES', 'VARA DE QUART', 'SANT ISIDRE',
       'TRINITAT', 'LA RAIOSA', 'PATRAIX', 'LA PETXINA', 'TRES FORQUES',
       'LA FONTSANTA', "L'ILLA PERDUDA", 'ELS ORRIOLS', 'EL GRAU',
       'LA XEREA', 'NA ROVELLA', 'SAFRANAR', 'CIUTAT JARDI',
       'LA FONTETA S.LLUIS', 'SANT MARCEL.LI', 'NATZARET', 'EN CORTS',
       'CIUTAT FALLERA', 'LA SEU', 'EL MERCAT', 'LA VEGA BAIXA',
       'BENIFERRI', "L'AMISTAT", 'FAVARA', "L'HORT DE SENABRE",
       'BENIMAMET', 'CAMI REAL', 'MORVEDRE', 'EL BOTANIC', 'TORMOS',
       'CAMI DE VERA', 'LA CREU DEL GRAU', 'LA ROQ

In [38]:
# Recuento de registros por estación
df2[var_analisis].value_counts()
# Vemos si hay variaciones en toda la muestra
df2[var_analisis].value_counts().describe()

count     70.0
mean     673.0
std        0.0
min      673.0
25%      673.0
50%      673.0
75%      673.0
max      673.0
Name: count, dtype: float64

In [39]:
# Conversión de la columna updated
df2["fecha"] = pd.to_datetime(df2.fecha)
# Rango temporal de la muestra
print(df2["fecha"].min(), df2["fecha"].max())

2025-02-03 00:00:00+00:00 2025-03-03 10:00:00+00:00


In [40]:
# Descripción de todas las variables disponibles
df2.describe(include="all")

,prec,vmax,vv,hr,ta,tpr,tamin,tamax,fecha,uso_bici,name
count,47110.000000,47110.000000,47110.000000,47110.000000,47110.000000,47110.000000,47110.000000,47110.000000,47110,47110.000000,47110
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LA CARRASCA
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,673
mean,0.043536,2.143239,0.680535,63.583952,13.174443,5.984250,12.719019,13.655423,2025-02-17 01:33:36.641902080+00:00,5.284143,NaN
min,0.000000,0.000000,0.000000,26.000000,5.400000,-1.700000,5.300000,5.600000,2025-02-03 00:00:00+00:00,0.000000,NaN
25%,0.000000,0.000000,0.000000,54.000000,11.300000,3.200000,10.900000,11.800000,2025-02-10 00:00:00+00:00,1.000000,NaN
50%,0.000000,1.100000,0.000000,67.000000,13.300000,6.200000,12.900000,13.600000,2025-02-17 00:00:00+00:00,3.000000,NaN
75%,0.000000,3.600000,1.200000,76.000000,15.200000,8.900000,14.800000,15.800000,2025-02-24 00:00:00+00:00,7.000000,NaN
max,15.600000,19.600000,4.800000,81.000000,22.100000,11.300000,21.300000,22.700000,2025-03-03 10:00:00+00:00,122.000000,NaN


In [61]:
# Muestra por barrios el uso de bicicleta medio y desviación
df2.groupby(by=var_analisis).agg(av_mean=pd.NamedAgg(column="uso_bici", aggfunc="mean"),
                            av_std=pd.NamedAgg(column="uso_bici", aggfunc="std"))\
                            .sort_values("av_mean", ascending=False)

,av_mean,av_std
name,,
LA CARRASCA,25.595840,24.857872
CABANYAL-CANYAMELAR,18.997028,13.436358
AIORA,15.136701,11.627619
MESTALLA,13.219911,9.030821
RUSSAFA,12.989599,8.583529
...,...,...
LA LLUM,0.701337,0.931404
LA PUNTA,0.671620,1.428050
CIUTAT FALLERA,0.643388,0.968563


In [65]:
# Pivot table es útil para hacer agrupaciones con distintas variables categóricas
df2.pivot_table(index='name', columns=df2.fecha.dt.hour, values='uso_bici', aggfunc='mean')

fecha,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
name,,,,,,,,,,,,,,,,,,,,,
AIORA,4.392857,3.321429,1.642857,1.178571,1.714286,1.571429,4.068966,22.206897,33.928571,23.000000,...,22.607143,21.000000,21.464286,20.464286,19.571429,19.821429,17.857143,13.857143,12.571429,8.285714
ALBORS,5.892857,3.250000,1.928571,2.071429,1.000000,1.214286,2.551724,11.896552,27.285714,15.655172,...,11.678571,10.750000,10.571429,11.035714,11.714286,10.607143,9.321429,6.464286,5.678571,5.500000
ARRANCAPINS,6.142857,4.071429,3.142857,3.000000,3.750000,3.250000,5.482759,11.620690,16.964286,13.620690,...,14.928571,12.642857,14.571429,15.964286,15.571429,15.428571,15.535714,10.321429,8.714286,10.607143
BENICALAP,3.357143,1.821429,1.357143,0.678571,0.571429,1.642857,2.275862,10.620690,12.607143,7.758621,...,11.750000,8.928571,11.678571,12.714286,13.500000,11.071429,8.000000,5.107143,3.964286,3.821429
BENIFERRI,0.785714,0.107143,0.035714,0.035714,0.178571,0.250000,0.137931,0.517241,1.214286,1.482759,...,0.607143,1.321429,1.714286,2.000000,1.464286,1.357143,0.821429,0.392857,0.285714,0.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TORMOS,0.678571,0.642857,0.250000,0.142857,0.535714,0.857143,1.034483,6.310345,4.821429,4.275862,...,3.642857,3.607143,3.107143,4.142857,3.357143,3.107143,1.928571,2.928571,3.285714,1.214286
TORREFIEL,1.785714,0.535714,0.392857,0.142857,0.285714,2.107143,2.586207,6.655172,8.964286,6.965517,...,6.000000,4.357143,6.821429,5.714286,6.464286,5.892857,4.607143,4.500000,2.357143,2.285714
TRES FORQUES,0.428571,0.178571,0.250000,0.107143,0.071429,0.250000,0.275862,2.137931,2.785714,2.172414,...,4.000000,2.892857,1.428571,3.071429,2.714286,2.321429,1.892857,1.857143,1.214286,0.607143


In [66]:
# Estilos para visualización de tablas
headers = {
    'selector': 'th.col_heading',
    'props': 'background-color: #5E17EB; color: white;'
}

index_style = {
    'selector': 'th.index_name',
    'props': 'background-color: #5E17EB; color: white;'
}

In [64]:
df2.pivot_table(index=df2.fecha.dt.hour, columns='name', values='uso_bici').round(2)\
.style.set_table_styles([headers, index_style])\
.background_gradient(cmap='viridis')

name,AIORA,ALBORS,ARRANCAPINS,BENICALAP,BENIFERRI,BENIMACLET,BENIMAMET,CABANYAL-CANYAMELAR,CAMI DE VERA,CAMI FONDO,CAMI REAL,CAMPANAR,CIUTAT DE LES ARTS I DE LES CIENCIES,CIUTAT FALLERA,CIUTAT JARDI,CIUTAT UNIVERSITARIA,EL BOTANIC,EL CALVARI,EL CARME,EL GRAU,EL MERCAT,EL PLA DEL REMEI,ELS ORRIOLS,EN CORTS,EXPOSICIO,FAVARA,JAUME ROIG,L'AMISTAT,L'HORT DE SENABRE,L'ILLA PERDUDA,LA CARRASCA,LA CREU COBERTA,LA CREU DEL GRAU,LA FONTETA S.LLUIS,LA FONTSANTA,LA GRAN VIA,LA LLUM,LA MALVA-ROSA,LA PETXINA,LA PUNTA,LA RAIOSA,LA ROQUETA,LA SEU,LA VEGA BAIXA,LA XEREA,LES TENDETES,MALILLA,MARXALENES,MESTALLA,MONTOLIVET,MORVEDRE,NA ROVELLA,NATZARET,NOU MOLES,PATRAIX,PENYA-ROJA,RUSSAFA,SAFRANAR,SANT ANTONI,SANT FRANCESC,SANT ISIDRE,SANT LLORENS,SANT MARCEL.LI,SANT PAU,SOTERNES,TORMOS,TORREFIEL,TRES FORQUES,TRINITAT,VARA DE QUART
fecha,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4.390000,5.890000,6.140000,3.360000,0.790000,9.360000,0.000000,10.790000,1.680000,0.430000,0.210000,4.860000,3.960000,0.070000,7.610000,4.000000,2.250000,0.500000,8.320000,1.860000,3.540000,12.500000,1.640000,0.610000,3.500000,0.790000,0.710000,6.180000,0.500000,2.000000,7.570000,0.360000,1.610000,0.250000,0.390000,12.460000,0.210000,1.500000,1.570000,0.140000,2.320000,5.430000,4.540000,3.320000,3.710000,0.710000,1.860000,1.890000,12.820000,2.000000,1.390000,0.930000,0.250000,2.000000,1.290000,6.040000,13.000000,0.610000,0.430000,8.070000,0.360000,1.820000,0.290000,2.000000,0.140000,0.680000,1.790000,0.430000,4.070000,0.710000
1,3.320000,3.250000,4.070000,1.820000,0.110000,4.210000,0.000000,5.070000,0.540000,0.390000,0.250000,1.860000,3.540000,0.070000,5.070000,2.210000,1.210000,0.360000,6.290000,0.570000,2.110000,6.110000,0.640000,0.320000,1.360000,0.040000,0.290000,3.390000,0.360000,1.110000,4.610000,0.290000,0.460000,0.140000,0.210000,7.390000,0.110000,0.360000,1.070000,0.290000,1.610000,1.790000,1.430000,1.610000,2.000000,0.360000,0.430000,1.430000,7.570000,1.640000,1.210000,0.360000,0.110000,1.390000,1.180000,3.110000,6.960000,0.180000,0.250000,3.890000,0.360000,0.680000,0.070000,0.540000,0.070000,0.640000,0.540000,0.180000,2.320000,0.070000
2,1.640000,1.930000,3.140000,1.360000,0.040000,2.820000,0.040000,3.710000,0.430000,0.110000,0.040000,1.040000,2.320000,0.110000,3.930000,1.290000,0.540000,0.070000,3.680000,0.930000,1.320000,3.610000,0.290000,0.110000,0.860000,0.040000,0.070000,3.820000,0.140000,0.750000,2.750000,0.000000,0.290000,0.210000,0.110000,5.390000,0.070000,0.540000,0.500000,0.000000,1.250000,1.540000,1.290000,1.750000,1.040000,0.290000,0.460000,1.460000,5.960000,0.680000,0.500000,0.140000,0.000000,0.790000,0.250000,2.250000,5.320000,0.110000,0.140000,2.140000,0.320000,0.250000,0.040000,0.210000,0.070000,0.250000,0.390000,0.250000,1.040000,0.040000
3,1.180000,2.070000,3.000000,0.680000,0.040000,1.610000,0.040000,2.460000,0.040000,0.000000,0.040000,0.570000,2.460000,0.040000,3.820000,0.860000,0.250000,0.180000,2.180000,0.390000,1.430000,2.790000,0.250000,0.140000,0.930000,0.320000,0.180000,2.960000,0.140000,0.500000,1.320000,0.070000,0.290000,0.040000,0.000000,4.710000,0.210000,0.570000,0.290000,0.000000,2.680000,1.890000,0.930000,1.070000,0.750000,0.290000,0.360000,0.540000,3.750000,0.610000,0.790000,0.070000,0.070000,0.360000,0.290000,1.960000,3.040000,0.140000,0.040000,1.610000,0.180000,0.290000,0.140000,0.430000,0.000000,0.140000,0.140000,0.110000,1.320000,0.680000
4,1.710000,1.000000,3.750000,0.570000,0.180000,0.610000,0.000000,2.110000,0.180000,0.140000,0.000000,0.460000,1.680000,0.000000,2.110000,0.960000,0.390000,0.250000,1.570000,0.540000,0.820000,1.960000,0.680000,0.140000,0.610000,0.070000,0.250000,1.540000,0.180000,0.430000,1.210000,0.000000,0.180000,0.180000,0.140000,2.820000,0.000000,0.500000,0.390000,0.000000,1.430000,1.110000,0.390000,0.540000,0.500000,0.390000,1.040000,0.430000,2.180000,1.960000,0.820000,0.110000,0.000000,0.430000,0.110000,2.610000,2.750000,0.040000,0.110000,1.820000,0.040000,0.320000,0.82000

In [46]:
# Combinación de tablas con visualizaciones para resaltar tendencias
df2.pivot_table(index=df2.fecha.dt.hour, columns='name', values='uso_bici').round(2).abs()\
    .style.set_table_styles([headers, index_style])\
    .set_properties(**{'background-color': 'green', 'color': 'green'})\
    .bar(color='#BEEAE5')

name,AIORA,ALBORS,ARRANCAPINS,BENICALAP,BENIFERRI,BENIMACLET,BENIMAMET,CABANYAL-CANYAMELAR,CAMI DE VERA,CAMI FONDO,CAMI REAL,CAMPANAR,CIUTAT DE LES ARTS I DE LES CIENCIES,CIUTAT FALLERA,CIUTAT JARDI,CIUTAT UNIVERSITARIA,EL BOTANIC,EL CALVARI,EL CARME,EL GRAU,EL MERCAT,EL PLA DEL REMEI,ELS ORRIOLS,EN CORTS,EXPOSICIO,FAVARA,JAUME ROIG,L'AMISTAT,L'HORT DE SENABRE,L'ILLA PERDUDA,LA CARRASCA,LA CREU COBERTA,LA CREU DEL GRAU,LA FONTETA S.LLUIS,LA FONTSANTA,LA GRAN VIA,LA LLUM,LA MALVA-ROSA,LA PETXINA,LA PUNTA,LA RAIOSA,LA ROQUETA,LA SEU,LA VEGA BAIXA,LA XEREA,LES TENDETES,MALILLA,MARXALENES,MESTALLA,MONTOLIVET,MORVEDRE,NA ROVELLA,NATZARET,NOU MOLES,PATRAIX,PENYA-ROJA,RUSSAFA,SAFRANAR,SANT ANTONI,SANT FRANCESC,SANT ISIDRE,SANT LLORENS,SANT MARCEL.LI,SANT PAU,SOTERNES,TORMOS,TORREFIEL,TRES FORQUES,TRINITAT,VARA DE QUART
fecha,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,4.390000,5.890000,6.140000,3.360000,0.790000,9.360000,0.000000,10.790000,1.680000,0.430000,0.210000,4.860000,3.960000,0.070000,7.610000,4.000000,2.250000,0.500000,8.320000,1.860000,3.540000,12.500000,1.640000,0.610000,3.500000,0.790000,0.710000,6.180000,0.500000,2.000000,7.570000,0.360000,1.610000,0.250000,0.390000,12.460000,0.210000,1.500000,1.570000,0.140000,2.320000,5.430000,4.540000,3.320000,3.710000,0.710000,1.860000,1.890000,12.820000,2.000000,1.390000,0.930000,0.250000,2.000000,1.290000,6.040000,13.000000,0.610000,0.430000,8.070000,0.360000,1.820000,0.290000,2.000000,0.140000,0.680000,1.790000,0.430000,4.070000,0.710000
1,3.320000,3.250000,4.070000,1.820000,0.110000,4.210000,0.000000,5.070000,0.540000,0.390000,0.250000,1.860000,3.540000,0.070000,5.070000,2.210000,1.210000,0.360000,6.290000,0.570000,2.110000,6.110000,0.640000,0.320000,1.360000,0.040000,0.290000,3.390000,0.360000,1.110000,4.610000,0.290000,0.460000,0.140000,0.210000,7.390000,0.110000,0.360000,1.070000,0.290000,1.610000,1.790000,1.430000,1.610000,2.000000,0.360000,0.430000,1.430000,7.570000,1.640000,1.210000,0.360000,0.110000,1.390000,1.180000,3.110000,6.960000,0.180000,0.250000,3.890000,0.360000,0.680000,0.070000,0.540000,0.070000,0.640000,0.540000,0.180000,2.320000,0.070000
2,1.640000,1.930000,3.140000,1.360000,0.040000,2.820000,0.040000,3.710000,0.430000,0.110000,0.040000,1.040000,2.320000,0.110000,3.930000,1.290000,0.540000,0.070000,3.680000,0.930000,1.320000,3.610000,0.290000,0.110000,0.860000,0.040000,0.070000,3.820000,0.140000,0.750000,2.750000,0.000000,0.290000,0.210000,0.110000,5.390000,0.070000,0.540000,0.500000,0.000000,1.250000,1.540000,1.290000,1.750000,1.040000,0.290000,0.460000,1.460000,5.960000,0.680000,0.500000,0.140000,0.000000,0.790000,0.250000,2.250000,5.320000,0.110000,0.140000,2.140000,0.320000,0.250000,0.040000,0.210000,0.070000,0.250000,0.390000,0.250000,1.040000,0.040000
3,1.180000,2.070000,3.000000,0.680000,0.040000,1.610000,0.040000,2.460000,0.040000,0.000000,0.040000,0.570000,2.460000,0.040000,3.820000,0.860000,0.250000,0.180000,2.180000,0.390000,1.430000,2.790000,0.250000,0.140000,0.930000,0.320000,0.180000,2.960000,0.140000,0.500000,1.320000,0.070000,0.290000,0.040000,0.000000,4.710000,0.210000,0.570000,0.290000,0.000000,2.680000,1.890000,0.930000,1.070000,0.750000,0.290000,0.360000,0.540000,3.750000,0.610000,0.790000,0.070000,0.070000,0.360000,0.290000,1.960000,3.040000,0.140000,0.040000,1.610000,0.180000,0.290000,0.140000,0.430000,0.000000,0.140000,0.140000,0.110000,1.320000,0.680000
4,1.710000,1.000000,3.750000,0.570000,0.180000,0.610000,0.000000,2.110000,0.180000,0.140000,0.000000,0.460000,1.680000,0.000000,2.110000,0.960000,0.390000,0.250000,1.570000,0.540000,0.820000,1.960000,0.680000,0.140000,0.610000,0.070000,0.250000,1.540000,0.180000,0.430000,1.210000,0.000000,0.180000,0.180000,0.140000,2.820000,0.000000,0.500000,0.390000,0.000000,1.430000,1.110000,0.390000,0.540000,0.500000,0.390000,1.040000,0.430000,2.180000,1.960000,0.820000,0.110000,0.000000,0.430000,0.110000,2.610000,2.750000,0.040000,0.110000,1.820000,0.040000,0.320000,0.82000

## Estudio por barrio

In [ ]:
# Filtrado de datos usando .loc
df2.loc[df2['name']=='RUSSAFA', :]

,prec,vmax,vv,hr,ta,tpr,tamin,tamax,fecha,uso_bici,name
4038,0.0,2.5,0.9,72.0,6.5,1.9,6.4,7.1,2025-02-03 00:00:00+00:00,0.0,RUSSAFA
4039,0.0,2.1,0.5,71.0,6.3,1.5,6.2,6.6,2025-02-03 01:00:00+00:00,3.0,RUSSAFA
4040,0.0,2.5,1.1,71.0,6.0,1.1,6.0,6.3,2025-02-03 02:00:00+00:00,0.0,RUSSAFA
4041,0.0,2.6,0.4,70.0,5.8,0.9,5.8,6.2,2025-02-03 03:00:00+00:00,1.0,RUSSAFA
4042,0.0,2.1,0.5,70.0,5.7,0.6,5.7,5.9,2025-02-03 04:00:00+00:00,0.0,RUSSAFA
...,...,...,...,...,...,...,...,...,...,...,...
4706,0.0,8.6,2.0,81.0,14.1,10.9,13.7,14.1,2025-03-03 06:00:00+00:00,3.0,RUSSAFA
4707,0.0,9.2,2.1,81.0,14.3,11.1,14.1,14.4,2025-03-03 07:00:00+00:00,19.0,RUSSAFA
4708,0.6,9.3,2.2,81.0,14.1,10.9,14.1,14.4,2025-03-03 08:00:00+00:00,18.0,RUSSAFA
4709,1.5,8.0,2.3,81.0,14.3,11.1,14.0,14.4,2025-03-03 09:00:00+00:00,13.0,RUSSAFA


In [68]:
df2.loc[df2['name']=='RUSSAFA', :]\
    .pivot_table(index=df2.fecha.dt.hour, columns='name', values='uso_bici').round(2).abs()\
    .style.set_table_styles([headers, index_style])\
    .set_properties(**{'background-color': 'green', 'color': 'green'})\
    .bar(color='#BEEAE5')

name,RUSSAFA
fecha,
0,13.000000
1,6.960000
2,5.320000
3,3.040000
4,2.750000
5,1.430000
6,4.140000
7,18.690000
8,24.890000


In [74]:
# Filtra para comparar algunos de los barrios 
lista_filtro = ["RUSSAFA", "CIUTAT UNIVERSITARIA"]
df2.loc[df2['name'].isin(lista_filtro), :]\
    .pivot_table(index=df2.fecha.dt.hour, columns='name', values=['uso_bici', 'tamin'], aggfunc='mean')\
    .round(2).abs()\
    .style.set_table_styles([headers, index_style])\
    .set_properties(**{'background-color': 'green', 'color': 'green'})\
    .bar(color='#BEEAE5')


# Mejorar procesos de análisis

Importante ver la actividad de github:

https://github.com/ydataai/ydata-profiling
```
pip install ydata-profiling
```
[Documentación profiling](https://docs.profiling.ydata.ai/latest/)
```
pip install dtale
```

Alternativas:
- https://github.com/fbdesignpro/sweetviz
- https://github.com/AutoViML/AutoViz

In [ ]:
# Se pueden ver los datos desde el cuaderno, como muestras...
df2.sample()

In [ ]:
import dtale
from ydata_profiling import ProfileReport

In [ ]:
profile = ProfileReport(df2, title="Profiling Report")

profile.to_file("./documentos/informe_bicis.html")

## Dtale

Crear el fichero en csv:
```
df2.to_csv('./data/usobici.csv', index=False)
```
Error en versiones de python 3.12

En terminal:
```
df = pd.read_csv('./data/usobici.csv')
df["fecha"] = pd.to_datetime(df.fecha)
d = dtale.show(df)
d.open_browser()
```

# Conclusiones
- Se ha analizado el dataset
- ¿Cuáles son las ventajas de utilizar Python respecto a otros sistemas (excel, tableau,...)?
- ¿Qué aspectos pueden mejorarse al hacer un EDA con Jupyter notebook?
- ¿Qué otras preguntas de análisis exploratorio podrían incluirse en este apartado?